In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
sns.set(style="white", color_codes=True)
sns.set_context(rc={'font.family': 'sans', 'font.size': 24, 'axes.titlesize':24, 'axes.labelsize':24})
#if you want to know current working dir
os.chdir('../..')

from src.utils import *
from src.utility import merge_speed_events
import src.data as data
import src.utility as utils
from src.preprocessing.other_features import avg_speed_for_roadtype_event

%matplotlib inline

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import RegressorChain
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb

In [3]:
X, Y = data.dataset()

caching resources/dataset/preprocessed/local/train/merged_dataset.csv.gz
performing onehot
EMERGENCY_LANE
LANES
ROAD_TYPE
EVENT_DETAIL
EVENT_TYPE
SPEED_AVG_-4
SPEED_AVG_-3
SPEED_AVG_-2
SPEED_AVG_-1
SPEED_SD_-4
SPEED_SD_-3
SPEED_SD_-2
SPEED_SD_-1
SPEED_MAX_-4
SPEED_MAX_-3
SPEED_MAX_-2
SPEED_MAX_-1
SPEED_MIN_-4
SPEED_MIN_-3
SPEED_MIN_-2
SPEED_MIN_-1
N_VEHICLES_-4
N_VEHICLES_-3
N_VEHICLES_-2
N_VEHICLES_-1
WEATHER_-4
WEATHER_-3
WEATHER_-2
WEATHER_-1
DISTANCE_-4
DISTANCE_-3
DISTANCE_-2
DISTANCE_-1
TEMPERATURE_-4
TEMPERATURE_-3
TEMPERATURE_-2
TEMPERATURE_-1
MIN_TEMPERATURE_-4
MIN_TEMPERATURE_-3
MIN_TEMPERATURE_-2
MIN_TEMPERATURE_-1
MAX_TEMPERATURE_-4
MAX_TEMPERATURE_-3
MAX_TEMPERATURE_-2
MAX_TEMPERATURE_-1
avg_speed_street
avg_speed_sd_street
avg_speed_min_street
avg_speed_max_street
avg_n_vehicles_street
avg_speed_sensor
avg_speed_sd_sensor
avg_speed_min_sensor
avg_speed_max_sensor
avg_n_vehicles_sensor
avg_speed_sensor_hour_y_0
avg_speed_sd_sensor_hour_y_0
avg_speed_min_sensor_hour_y_0
avg

In [4]:
Y.head(2)

,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3
12804,NaN,NaN,NaN,NaN
4185,NaN,NaN,NaN,NaN


In [5]:
dset = pd.concat([X, Y], axis=1)

In [6]:
dset.dropna(inplace=True, subset=['SPEED_AVG_Y_0', 'SPEED_AVG_Y_1', 'SPEED_AVG_Y_2', 'SPEED_AVG_Y_3'])
dset.shape

(6394, 264)

In [76]:
Y = dset[['SPEED_AVG_Y_0', 'SPEED_AVG_Y_1', 'SPEED_AVG_Y_2', 'SPEED_AVG_Y_3']]
X = dset.drop(['SPEED_AVG_Y_0', 'SPEED_AVG_Y_1', 'SPEED_AVG_Y_2', 'SPEED_AVG_Y_3'], axis=1)
imputer = SimpleImputer().fit(X)
X_imputed = imputer.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_imputed, Y, test_size=0.20, shuffle=False)

In [72]:
# model = GradientBoostingRegressor(loss='lad', learning_rate=0.02, n_estimators=500, max_depth=3)
# model = LinearRegression(n_jobs=6)
# model = SVR(degree=5, cache_size=2000, max_iter=10)
# model = BayesianRidge(verbose=True)
base_model = GradientBoostingRegressor(loss='lad', criterion='mae', n_estimators=100, verbose=1)
ensambled_model = BaggingRegressor(base_estimator=base_model, n_estimators=8, max_samples=0.8, n_jobs=8, verbose=1).fit(X_train, y_train)
# model = RegressorChain(base_estimator=ensambled_model).fit(X_train, y_train)

/Users/gianpaolo/anaconda3/envs/dmtm/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:397: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed: 12.7min remaining: 38.0min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed: 13.1min finished


In [78]:
mean_absolute_error(model.predict(X_test), y_test)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished


9.410892806954072

In [75]:
model.predict(X_test)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished


array([[183.2858988 , 182.32008165, 180.86903435, 176.99972419],
       [175.36661068, 171.62956809, 170.5854976 , 168.50135857],
       [106.53241414, 110.04673479, 108.89366853, 116.11275602],
       ...,
       [165.80645382, 175.13334187, 183.09039662, 188.27643883],
       [194.3612005 , 194.24850561, 195.10160707, 194.90050583],
       [179.41757589, 177.03565286, 175.76123404, 175.32323942]])